In [ ]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
# pd.set_option('display.max_rows', 100)
# pd.set_option('display.max_seq_items', 100)

# 400 meter coordinate accuracy
# 1923 earliest year

In [ ]:
# read in new_old merged data
new_path = input("Enter the file path: ")

new = pd.read_parquet(new_path)

In [ ]:
# read in wwf clean data
wwf_path = input("Enter the file path: ")

# read in the old species data in json format
wwf = pd.read_parquet(wwf_path)

In [ ]:
wwf


In [ ]:
# rename usda column species to scientific_name
wwf.rename(columns={"species": "scientific_name"}, inplace=True)

In [ ]:
inner = pd.merge(new, wwf, how="inner", on=["scientific_name"], indicator=True)

In [ ]:
inner


In [ ]:
inner["combined_id"] = inner["unique_id_x"].apply(lambda x: x.tolist()) + inner[
    "unique_id_y"
].apply(lambda x: x.tolist())


In [ ]:
inner["combined_id"] = inner["combined_id"].apply(set).apply(list)


In [ ]:
check_combined_id = inner[["unique_id_x", "unique_id_y", "combined_id"]]


In [ ]:
inner = inner.drop(["unique_id_x", "unique_id_y"], axis=1)

inner.rename(columns={"combined_id": "unique_id"}, inplace=True)

del check_combined_id


In [ ]:
inner


In [ ]:
# keep columns with less missing values


# find sum of missing values in inner['class_x']
class_x_missing = inner["class_x"].isna().sum()

class_y_missing = inner["class_y"].isna().sum()

# drop the class column from inner that contains more missing values
if class_x_missing > class_y_missing:
    inner = inner.drop(["class_x"], axis=1)
    inner.rename(columns={"class_y": "class"}, inplace=True)
else:
    inner = inner.drop(["class_y"], axis=1)
    inner.rename(columns={"class_x": "class"}, inplace=True)

# check
class_missing = inner["class"].isna().sum()

order_x_missing = inner["order_x"].isna().sum()

order_y_missing = inner["order_y"].isna().sum()

# drop the order column from inner that contains more missing values

if order_x_missing > order_y_missing:
    inner = inner.drop(["order_x"], axis=1)
    inner.rename(columns={"order_y": "order"}, inplace=True)
else:
    inner = inner.drop(["order_y"], axis=1)
    inner.rename(columns={"order_x": "order"}, inplace=True)

family_x_missing = inner["family_x"].isna().sum()

family_y_missing = inner["family_y"].isna().sum()

# drop the family column from inner that contains more missing values

if family_x_missing > family_y_missing:
    inner = inner.drop(["family_x"], axis=1)
    inner.rename(columns={"family_y": "family"}, inplace=True)
else:
    inner = inner.drop(["family_y"], axis=1)
    inner.rename(columns={"family_x": "family"}, inplace=True)

In [ ]:
del (
    class_missing,
    order_x_missing,
    order_y_missing,
    family_x_missing,
    family_y_missing,
    class_x_missing,
    class_y_missing,
)

In [ ]:
inner

In [ ]:
common_name_x_missing = inner["common_name_x"].isna().sum()

common_name_y_missing = inner["common_name_y"].isna().sum()

In [ ]:
if common_name_x_missing > common_name_y_missing:
    inner = inner.drop(["common_name_x"], axis=1)
    inner.rename(columns={"common_name_y": "common_name"}, inplace=True)
else:
    inner = inner.drop(["common_name_y"], axis=1)
    inner.rename(columns={"common_name_x": "common_name"}, inplace=True)

del common_name_x_missing, common_name_y_missing

In [ ]:
inner = inner.drop(["_merge"], axis=1)


In [ ]:
inner = inner[
    [
        "kingdom",
        "phylum",
        "class",
        "order",
        "family",
        "genus",
        "scientific_name",
        "common_name",
        "species_type",
        "unique_id",
        "rights",
    ]
]


In [ ]:
outer = pd.merge(new, wwf, how="outer", on=["scientific_name"], indicator=True)


In [ ]:
outer

In [ ]:
left = outer.loc[
    outer._merge == "left_only",
    [
        "kingdom",
        "phylum",
        "class_x",
        "order_x",
        "family_x",
        "genus",
        "scientific_name",
        "common_name_x",
        "species_type",
        "unique_id_x",
        "rights",
    ],
]

In [ ]:
left.rename(
    columns={
        "class_x": "class",
        "order_x": "order",
        "family_x": "family",
        "unique_id_x": "unique_id",
        "common_name_x": "common_name",
    },
    inplace=True,
)

In [ ]:
left = left[
    [
        "kingdom",
        "phylum",
        "class",
        "order",
        "family",
        "genus",
        "scientific_name",
        "common_name",
        "species_type",
        "unique_id",
        "rights",
    ]
]

In [ ]:
right = outer.loc[
    outer._merge == "right_only",
    [
        "kingdom",
        "phylum",
        "class_y",
        "order_y",
        "family_y",
        "genus",
        "scientific_name",
        "common_name_y",
        "species_type",
        "unique_id_y",
        "rights",
    ],
]

In [ ]:
right.rename(
    columns={
        "class_y": "class",
        "order_y": "order",
        "family_y": "family",
        "unique_id_y": "unique_id",
        "common_name_y": "common_name",
    },
    inplace=True,
)

In [ ]:
right = right[
    [
        "kingdom",
        "phylum",
        "class",
        "order",
        "family",
        "genus",
        "scientific_name",
        "common_name",
        "species_type",
        "unique_id",
        "rights",
    ]
]

In [ ]:
frames = [inner, left, right]

# concatenate dataframes
final = pd.concat(frames)

# reset index
final.reset_index(drop=True, inplace=True)

In [ ]:
final


In [ ]:
# find unique values in scientific_name column of final
species_unique = final["scientific_name"].unique()

In [ ]:
# read cleaned df into a parquet file where user can input the file path
df_merge_path = input("Enter the file path: ")


# write df to parquet file using pandas to_parquet
final.to_parquet(df_merge_path)
